#Comet Demo

This demo requires the comet package and the requests package.  The requests package comes with Anaconda, but you will have to pip install the comet package.<br>
`$> pip install comet`

##The broker

First spin up the broker. The broker is a comet instance operating in a read/broadcast mode.  This allows for users to post to the broker for further filtering and dissemination.  Subscribers can register with this broker.

In [33]:
import tempfile, os, subprocess

broker_dir = tempfile.mkdtemp()

#start up a broker and save the pid so we can kill it
              #receive and send
command_str = "twistd comet --eventdb=%s --receive --broadcast "%(broker_dir)
               #Specify ports to operate on
command_str += "--receive-port=8808 --broadcast-port=8809 "
               #The plugin prints all events to the log.  Also need a ivorn.
command_str += "--print-event --local-ivo=ivo://me.mybroker/comet_broker"

#Move to a working directory so we can sandbox multiple comet instances on one box
os.chdir(broker_dir)
subprocess.call(command_str.split())

fh = open('twistd.pid')

broker_pid = int(fh.readline().rstrip())
print "Broker running in %s with PID=%i"%(broker_dir, broker_pid)

Broker running in /var/folders/q2/3j92svhj3g16lxf08v1yc3280000gn/T/tmp3BA8wp with PID=41235


I find it useful to watch the log file.  In some browsers (not Safari) you can do this by pointing your browser to `file://[path_from_above]/twistd.log`.

##The subscribers

The subscription process is handled handled by the small web application in this repository.

To run the web application, change to the top level directory of this git repository and run the following command:<br>
`$> python web_listener.py`

Now you can check out the web interface <a href="http://localhost:8880/"> here</a>.  <br>
Try making a new filter.  The default filter will pass all alerts on.  The broker address should be 'localhost:8809' unless you changed it in the code above.

Alerts are XPATH strings and if the XPATH evaluates to a node in the XML VOEvent will evaluate to true.  For the purpose of this demo, we will be using query strings like `//Who/Author[contactName="KSK"]`.  This query means "Is this alert packet sent by an entity who's contact name is KSK?"

##Generate and send alerts

In order to test the interface, we need a way to generate alerts.  The follwoing code generates "test" alerts that can be used to demonstrate the functionality of the system without infringing on the operations of normal alerts.

In [34]:
from random import randint
from functools import partial
def genAlertTxt(sender, outfile='testAlert.xml', idFactory=partial(randint, 10, 10000)):
    """Generate a test alert packet
    @param[in] sender  String indicating the name of the sender of this packet
    @param[in] outfile  Name of the file to store the output in
    @param[in] idFactory  A function used to generate unique ids for each alert
    """
    alertTmpl = """<?xml version='1.0' encoding='UTF-8'?>
 <voe:VOEvent xmlns:voe="http://www.ivoa.net/xml/VOEvent/v2.0"
     xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
     xsi:schemaLocation="http://www.ivoa.net/xml/VOEvent/v2.0
         http://www.ivoa.net/xml/VOEvent/VOEvent-v2.0.xsd"
         version="2.0" role="test" ivorn="ivo://me.testevent/test_event_stream#{event_id}">
     <Who>
         <AuthorIVORN>ivo://me.testevent/comet-author</AuthorIVORN>
         <Author>
             <shortName>Test Author</shortName>
             <contactName>{sender}</contactName>
         </Author>
         <Date>2012-01-01T00:00:00</Date>
     </Who>
 </voe:VOEvent>
    """
    event_id = idFactory()
    fh = open(outfile, 'w')
    fh.write(alertTmpl.format(sender=sender, event_id=event_id))
    fh.close()

Now try sending a few alerts.  This can be done with the comet-sendvo tool. First generate a test alert and then send it using the command line tool.  Note the port should correspond to the port on which your broker spawned above is listening.  Like so:

In [35]:
genAlertTxt('KSK')
!comet-sendvo --host=localhost --port=8808 < testAlert.xml

2015-05-31 17:59:25-0700 [-] Log opened.
2015-05-31 17:59:25-0700 [-] Starting factory <__main__.OneShotSender instance at 0x104f5ebd8>
2015-05-31 17:59:25-0700 [INFO VOEventSender,client] Acknowledgement received from IPv4Address(TCP, '127.0.0.1', 8808)
2015-05-31 17:59:25-0700 [VOEventSender,client] Stopping factory <__main__.OneShotSender instance at 0x104f5ebd8>
2015-05-31 17:59:25-0700 [INFO VOEventSender,client] Event was sent successfully
2015-05-31 17:59:25-0700 [-] Main loop terminated.


The above can be used to spawn several alerts.  Experiment with queries and the registry.